In [44]:
import pandas as pd

RSV = pd.read_csv("06_RSV_bez_chybnych_radku_M1.csv", sep=",", on_bad_lines="skip", quoting=0, encoding="utf-8", low_memory=False)

RSV.head(6)

,Datum 1 registrace,Datum 1 registrace v ČR,Druh vozidla,Kategorie vozidla,Tovární značka,Palivo,Plně elektrické vozidlo,Hybridní vozidlo,PČV,Rok výroby
0,01.01.1978,06.11.1989,OSOBNÍ AUTOMOBIL,M1,.,BA 96,NE,NE,3632073,1978
1,01.01.1963,26.06.1995,OSOBNÍ AUTOMOBIL,M1,.,BA 80,NE,NE,3632075,1963
2,27.05.1971,27.05.1971,OSOBNÍ AUTOMOBIL,M1,ŠKODA,BA 92 B,NE,NE,3632078,1971
3,01.01.1984,01.01.1984,OSOBNÍ AUTOMOBIL,M1,ŠKODA,BA 96,NE,NE,3632079,1984
4,01.01.1973,13.07.1992,OSOBNÍ AUTOMOBIL,M1,ŠKODA,BA 84,NE,NE,3632080,1973
5,20.04.1970,20.04.1970,OSOBNÍ AUTOMOBIL,M1,ŠKODA,BA 84,NE,NE,3632081,1970


##### všechny Tesly nastaveny na Palivo "EL"

In [45]:
RSV.loc[(RSV["Tovární značka"] == "TESLA") & (RSV["Palivo"] != "EL"), "Palivo"] = "EL"
RSV.loc[(RSV["Tovární značka"] == "TESLA MOTORS") & (RSV["Palivo"] != "EL"), "Palivo"] = "EL"

In [46]:
RSV.loc[RSV["Palivo"] == "ELEKTŘINA", "Palivo"] = "EL"
RSV.loc[RSV["Palivo"] == "EP ELEKTRICKÝ POHON", "Palivo"] = "EL"
RSV.loc[RSV["Palivo"] == "ELEKTRO", "Palivo"] = "EL"
RSV.loc[RSV["Palivo"] == "EL.", "Palivo"] = "EL"



In [47]:
RSV.loc[RSV["Palivo"] == "BA + EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "NM + EL", "Palivo"] = "EL + NM"
RSV.loc[RSV["Palivo"] == "EL + BA 95 B", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 95 B + EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA/EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 98 B + EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 95 B+EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "NM+EL", "Palivo"] = "EL + NM"
RSV.loc[RSV["Palivo"] == "BA+EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA95B+EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA95B+EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 91 B + EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 95 B / EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 95 B/ EL. ", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "BA 95 B; ELKTRO", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "NM+ELEKTROMOTOR", "Palivo"] = "EL + NM"
RSV.loc[RSV["Palivo"] == "BA SMĚS + EL", "Palivo"] = "EL + BA"
RSV.loc[RSV["Palivo"] == "NM/EL", "Palivo"] = "EL + NM"
RSV.loc[RSV["Palivo"] == "BA + EL + LPG", "Palivo"] = "EL + BA + LPG"
RSV.loc[RSV["Palivo"] == "BA 95 B+LPG+EL", "Palivo"] = "EL + BA + LPG"
RSV.loc[RSV["Palivo"] == "HYBR. BENZIN/E", "Palivo"] = "EL + BA"

##### bylo tam moc hodnot obsahující EL a kombinace EL a BA nebo NM, sjednoceno

In [48]:
RSV[RSV["Palivo"].str.contains("EL", na=False)]["Palivo"].unique()

array(['EL', 'EL + BA', 'EL + NM', 'EL + BA + LPG', 'BA + EL + E 85'],
      dtype=object)

##### tam, kde není obsaženo písmeno, jenom číslo, tak přejmenovat na "-"

In [49]:
RSV.loc[~RSV["Palivo"].str.contains(r"[a-zA-Z]", na=False), "Palivo"] = "-"

##### Najdi všechny řádky, kde sloupec Palivo obsahuje "BA", ale zároveň neobsahuje žádné z těchto výrazů: "LPG", "NM", "CNG", "E 85", "E85", "+", nebo "EL" — a nastav hodnotu Palivo na "BA"

In [50]:
RSV.loc[
    RSV["Palivo"].str.contains("BA", na=False) &
    ~RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("NM", na=False) &
    ~RSV["Palivo"].str.contains("CNG", na=False) &
    ~RSV["Palivo"].str.contains("E 85", na=False) &
    ~RSV["Palivo"].str.contains("E85", na=False) &
    # ~RSV["Palivo"].str.contains("\\+", na=False) &
    ~RSV["Palivo"].str.contains("EL", na=False),
    "Palivo"
] = "BA"

##### teď chci hodnoty, obsahující BA a LPG, ale ne CNG, E85 a EL, tak přejmenovat na BA + LPG a to samé NM a LPG, BA a CNG

In [51]:
RSV.loc[
    RSV["Palivo"].str.contains("BA", na=False) &
    RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("CNG", na=False) &
    ~RSV["Palivo"].str.contains("E85", na=False) &
    ~RSV["Palivo"].str.contains("E 85", na=False) &
    ~RSV["Palivo"].str.contains("EL", na=False),
    "Palivo"
# ]["Palivo"].unique()
] = "BA + LPG"

RSV.loc[
    RSV["Palivo"].str.contains("NM", na=False) &
    RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("CNG", na=False) &
    ~RSV["Palivo"].str.contains("E85", na=False) &
    ~RSV["Palivo"].str.contains("E 85", na=False) &
    ~RSV["Palivo"].str.contains("EL", na=False),
    "Palivo"
] = "NM + LPG"

RSV.loc[
    RSV["Palivo"].str.contains("BA", na=False) &
    RSV["Palivo"].str.contains("CNG", na=False) &
    ~RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("E85", na=False) &
    ~RSV["Palivo"].str.contains("E 85", na=False) &
    ~RSV["Palivo"].str.contains("EL", na=False),
    "Palivo"
] = "BA + CNG"

RSV.loc[
    RSV["Palivo"].str.contains("NM", na=False) &
    ~RSV["Palivo"].str.contains("BA", na=False) &
    ~RSV["Palivo"].str.contains("CNG", na=False) &
    ~RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("E85", na=False) &
    ~RSV["Palivo"].str.contains("E 85", na=False) &
    ~RSV["Palivo"].str.contains("EL", na=False),
    "Palivo"
] = "NM"
# ].unique()

In [52]:
RSV.loc[RSV["Palivo"] == "BA 95 B + E 85+ CNG", "Palivo"] = "BA + CNG + E85"
RSV.loc[RSV["Palivo"] == "CNG + LPG", "Palivo"] = "LPG + CNG"
RSV.loc[RSV["Palivo"] == "ba+CNG", "Palivo"] = "BA + CNG"
RSV.loc[RSV["Palivo"] == "BA + CNG + LPG", "Palivo"] = "BA + LPG + CNG"
RSV.loc[RSV["Palivo"] == "BA 91 B+CNG+LPG", "Palivo"] = "BA + LPG + CNG"
RSV.loc[RSV["Palivo"] == "BA SMĚS + E 85 + CNG", "Palivo"] = "BA + CNG + E85"
RSV.loc[RSV["Palivo"] == "CNG + LPG + BA", "Palivo"] = "BA + LPG + CNG"
RSV.loc[RSV["Palivo"] == "HYBR. BENZIN/E", "Palivo"] = "EL + BA"

In [53]:
RSV[RSV["Palivo"].str.contains("CNG", na=False)]["Palivo"].unique()

array(['BA + CNG', 'CNG', 'BA + LPG + CNG', 'BA + CNG + E85', 'LPG + CNG',
       'BA 95 B+LPG/CNG', 'BA + LPG/CNG', 'NM + CNG'], dtype=object)

In [54]:
# RSV[RSV["Palivo"].str.contains("90|91|95|96|98", na=False)]["Palivo"].unique()
RSV.loc[
    RSV["Palivo"].str.contains("90|91|95|96|98", na=False) &
    ~RSV["Palivo"].str.contains("LPG|E85|E 85", na=False),
    "Palivo"
] = "BA"

# 1. BA + LPG + E 85
RSV.loc[
    RSV["Palivo"].str.contains("90|91|95|96|98", na=False) &
    RSV["Palivo"].str.contains("LPG", na=False) &
    RSV["Palivo"].str.contains("E85|E 85", na=False),
    "Palivo"
] = "BA + LPG + E85"

# 2. BA + LPG
RSV.loc[
    RSV["Palivo"].str.contains("90|91|95|96|98", na=False) &
    RSV["Palivo"].str.contains("LPG", na=False) &
    ~RSV["Palivo"].str.contains("E85|E 85", na=False),
    "Palivo"
] = "BA + LPG"

# 3. BA + E85
RSV.loc[
    RSV["Palivo"].str.contains("90|91|95|96|98", na=False) &
    ~RSV["Palivo"].str.contains("LPG", na=False) &
    RSV["Palivo"].str.contains("E85|E 85", na=False),
    "Palivo"
] = "BA + E85"

In [55]:
RSV[RSV["Palivo"].str.contains("BA", na=False)]["Palivo"].unique()

array(['BA', 'BA + LPG', 'BA + CNG', 'BA + E 85', 'BA + E85', 'BA+E85',
       'BA SMĚS + E 85 + LPG', 'EL + BA', 'BA + E 85 + LPG',
       'BA + LPG + CNG', 'BA + CNG + E85', 'BA +E85', 'BA + LPG + E85',
       'BA  + E85', 'BA+E85+LPG', 'BA/E85', 'BA+E 85', 'BA NEBO E85',
       'BA SMĚS + LPG + E 85', 'BA +  E85', 'BA + E 85 + PB', 'E 85 + BA',
       'BA /E85', 'NM + BA', 'BA + E 85/LPG', 'BA/ E85', 'EL + BA + LPG',
       'BA +E 85', 'BA + LPG/CNG', 'BA  + E 85', 'BA + EL + E 85'],
      dtype=object)

In [57]:
RSV.loc[RSV["Palivo"] == "BA + E 85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA+E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA SMĚS + E 85 + LPG", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA + E 85 + LPG", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA +E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA + LPG + E 85", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA  + E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA+E85+LPG", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA/E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA+E 85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA SMĚS + LPG + E 85", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA +  E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "E 85 + BA", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA /E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA + E 85/LPG", "Palivo"] = "BA + LPG + E85"
RSV.loc[RSV["Palivo"] == "BA/ E85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA +E 85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA  + E 85", "Palivo"] = "BA + E85"
RSV.loc[RSV["Palivo"] == "BA + EL + E 85", "Palivo"] = "EL + BA + E85"
RSV.loc[RSV["Palivo"] == "BA + E 85 + PB", "Palivo"] = "BA + LPG + E85"

In [58]:
RSV[RSV["Palivo"].str.contains("BA", na=False)]["Palivo"].unique()

array(['BA', 'BA + LPG', 'BA + CNG', 'BA + E85', 'BA + LPG + E85',
       'EL + BA', 'BA + LPG + CNG', 'BA + CNG + E85', 'BA NEBO E85',
       'NM + BA', 'EL + BA + LPG', 'BA + LPG/CNG', 'EL + BA + E85'],
      dtype=object)

#### manuální výběr zařazení do správných skupin

In [59]:
RSV.loc[RSV["Palivo"] == "BB NATURAL", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZINOVÁ SMĚS", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "PB", "Palivo"] = "LPG"
RSV.loc[RSV["Palivo"] == "BENZ.SPECIÁL", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZÍN", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "NATURAL", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZÍNOVÁ SMĚS.", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "NAFTA", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "nafta", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "Benzin special", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZ.SMĚS", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "SUPER", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "NAFTA MOTOROVÁ", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "BN NATURAL", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZÍN BEZOLOVNATÝ", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "+LPG", "Palivo"] = "LPG"
RSV.loc[RSV["Palivo"] == "BENZIN", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZÍNOVÁ SMĚS", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "BENZIN NATURAL", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "ba 72", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "E 85", "Palivo"] = "E85"
RSV.loc[RSV["Palivo"] == "ZP ZEMNÍ PLYN STLAČE", "Palivo"] = "PLYN"
RSV.loc[RSV["Palivo"] == "nm", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "nm bio 48", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "2001/100A", "Palivo"] = "-"
RSV.loc[RSV["Palivo"] == "2003/76B", "Palivo"] = "-"
RSV.loc[RSV["Palivo"] == "- POLOŽKA NEBYLA NAP", "Palivo"] = "-"
RSV.loc[RSV["Palivo"] == "63 OČ", "Palivo"] = "-"
RSV.loc[RSV["Palivo"] == "83-02B", "Palivo"] = "-"
RSV.loc[RSV["Palivo"] == "BS 94", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "ZEMNÍ PLYN STLAČE", "Palivo"] = "CNG"
RSV.loc[RSV["Palivo"] == "PLYN", "Palivo"] = "LPG"
RSV.loc[RSV["Palivo"] == "NF", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "NN", "Palivo"] = "NM"
RSV.loc[RSV["Palivo"] == "BN", "Palivo"] = "BA"
RSV.loc[RSV["Palivo"] == "PB-BENZÍN/LPG/", "Palivo"] = "BA + LPG"
RSV.loc[RSV["Palivo"] == "PB/LPG", "Palivo"] = "LPG"


In [60]:
# pd.DataFrame(RSV["Palivo"].unique(), columns=["Unikátní hodnoty paliva"])

pd.set_option('display.max_rows', None) 
pd.DataFrame(RSV["Palivo"].unique(), columns=["Unikátní hodnoty paliva"])

,Unikátní hodnoty paliva
0,BA
1,NM
2,BA + LPG
3,E85
4,-
5,SP
6,BA + CNG
7,BA + E85
8,NP
9,EP


In [61]:
RSV.to_csv("06_RSV_palivo_EL.csv", index=False, encoding="utf-8")